In [1]:
text = 'I am testing my function with a short text message'

def count_words(text):
  """
  Count the number of times each word occurs in text (str).
  Return dictionary where keys are unique word and values
  are word counts
  """
  text = text.lower()    # turn every letter into lower case
  skip_list = [',', '.', ':', ';', '?', '!', '"', "'", "-"]
  for ch in skip_list:
    text = text.replace(ch, '')
  word_counts = {}
  for word in text.split(' '):
    if word in word_counts:
      word_counts[word] += 1
    else:
      word_counts[word] = 1
  return word_counts

count_words(text)

{'a': 1,
 'am': 1,
 'function': 1,
 'i': 1,
 'message': 1,
 'my': 1,
 'short': 1,
 'testing': 1,
 'text': 1,
 'with': 1}